In [69]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON


In [70]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-15,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-15,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-15,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-15,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-15,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [71]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7795


In [72]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 78
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [73]:
print(screener_df.head())
print()
print(len(screener_df.index))

  symbol     marketcap exchange                 exchangeName
0      A  3.948710e+10     XNYS  New York Stock Exchange Inc
1     AA  4.888346e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.961422e+07     XNAS           Nasdaq All Markets
3   AACI  2.587867e+07     XNAS           Nasdaq All Markets
4  AACIU  2.587867e+07     XNAS           Nasdaq All Markets

7795


In [74]:
# drop NAs
screener_df.dropna(inplace=True)

In [75]:
# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()
print(screener_df.head())

New length: 7688

  symbol     marketcap exchange                 exchangeName
0      A  3.948710e+10     XNYS  New York Stock Exchange Inc
1     AA  4.888346e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.961422e+07     XNAS           Nasdaq All Markets
3   AACI  2.587867e+07     XNAS           Nasdaq All Markets
4  AACIU  2.587867e+07     XNAS           Nasdaq All Markets


In [76]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

# apply function to 'marketcap' column AND create new 'marketcapType' column
screener_df['marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)
print(screener_df.head(10))

   symbol     marketcap exchange                 exchangeName marketcapType
0       A  3.948710e+10     XNYS  New York Stock Exchange Inc     Large-Cap
1      AA  4.888346e+09     XNYS  New York Stock Exchange Inc       Mid-Cap
2    AACG  4.961422e+07     XNAS           Nasdaq All Markets     Micro-Cap
3    AACI  2.587867e+07     XNAS           Nasdaq All Markets     Micro-Cap
4   AACIU  2.587867e+07     XNAS           Nasdaq All Markets     Micro-Cap
5    AACT  5.262500e+08     XNYS  New York Stock Exchange Inc     Small-Cap
7   AACT=  5.276897e+08     XNYS  New York Stock Exchange Inc     Small-Cap
8    AADI  4.659913e+07     XNAS           Nasdaq All Markets     Micro-Cap
9    AADR  2.569050e+07     XNAS           Nasdaq All Markets     Micro-Cap
10   AAGR  4.802947e+07     XNAS           Nasdaq All Markets     Micro-Cap


In [77]:
# # DEV - marketcap groupby count
# marketcap_counts = screener_df['marketcapType'].value_counts()

# # convert SERIES to DF
# marketcap_counts_df = marketcap_counts.reset_index()
# marketcap_counts_df.columns = ['marketcapType', 'count']

# # add percentage column
# total_count = marketcap_counts.sum()
# marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# # print
# marketcap_counts_df

In [78]:
# # DEV - checker
# total_count_CHECK = marketcap_counts_df['count'].sum()

# # if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
# if total_count_CHECK == len(screener_df):
#     print('Data lengths match! SUCCESS')
# else:    
#     print('Data mismatch! ERROR')
#     print()
#     print(f'Code checker: {total_count_CHECK}')
#     print(f'Data length: {len(screener_df)}')

In [79]:
screener_df.head(2)

,symbol,marketcap,exchange,exchangeName,marketcapType
0,A,3.948710e+10,XNYS,New York Stock Exchange Inc,Large-Cap
1,AA,4.888346e+09,XNYS,New York Stock Exchange Inc,Mid-Cap


In [80]:
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

quote_data_list = []

def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'latestPrice': quote_data.get('latestPrice', None),
            'previousClose': quote_data.get('previousClose', None), # PREV DAY CLOSE
            'extendedPrice': quote_data.get('extendedPrice', None), # EXTENDED HOURS PRICE
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), # EXTENDED HOURS CHANGE
            'extendedPriceTime': quote_data.get('extendedPriceTime', None), # EXTENDED HOURS PRICE TIME
            'latestVolume': quote_data.get('latestVolume', None), # LATEST VOLUME
            'companyName': quote_data.get('companyName', None) # COMPANY NAME
        })
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)

# Merge with the screener_df
screener_df = screener_df.merge(quote_df[['symbol',
                                        'latestPrice',
                                        'previousClose', #MOVED UP
                                        'extendedPrice', #NEW
                                        'extendedChangePercent', #NEW #IMPORTANT
                                        'extendedPriceTime', #NEW  
                                        'latestVolume', #NEW
                                        'companyName']], on='symbol', how='left')


Total batches: 77


In [86]:
# set the column order 
screener_df = screener_df[['symbol', # ticker symbol
                            'latestPrice',
                            'previousClose',
                            'extendedPrice',
                            'extendedChangePercent', 
                            'latestVolume',
                            'marketcapType',
                            'marketcap',
                            'exchange', 
                            'exchangeName', 
                            'extendedPriceTime', 
                            'companyName'
                            ]]

In [104]:
# print df with ordered cols 
screener_df.head(2)

,symbol,latestPrice,previousClose,extendedPrice,extendedChangePercent,latestVolume,marketcapType,marketcap,exchange,exchangeName,extendedPriceTime,companyName
0,A,134.75,132.24,134.75,0.0000,1514490,Large-Cap,3.948710e+10,XNYS,New York Stock Exchange Inc,1.708042e+12,Agilent Technologies Inc.
1,AA,27.39,26.79,27.36,-0.0011,4385488,Mid-Cap,4.888346e+09,XNYS,New York Stock Exchange Inc,1.708045e+12,Alcoa Corp


In [105]:
# create a filterd dataframe of selected stocks
# selected_stocks = screener_df[(screener_df['marketcapType'] == 'Large-Cap') & (screener_df['latestPrice'] > 100)]

# query for TSLA, PLTR, SNOW
selected_stocks = screener_df[(screener_df['symbol'] == 'TSLA') | (screener_df['symbol'] == 'PLTR') | (screener_df['symbol'] == 'SNOW')]
# fix index
selected_stocks.reset_index(drop=True, inplace=True)

In [106]:
# look at filterd dataframe
selected_stocks

,symbol,latestPrice,previousClose,extendedPrice,extendedChangePercent,latestVolume,marketcapType,marketcap,exchange,exchangeName,extendedPriceTime,companyName
0,PLTR,25.41,25.19,25.43,0.00079,70544321,Large-Cap,5.258441e+10,XNYS,New York Stock Exchange Inc,1.708045e+12,Palantir Technologies Inc - Class A
1,SNOW,232.40,236.00,232.63,0.00099,3617604,Large-Cap,7.652932e+10,XNYS,New York Stock Exchange Inc,1.708045e+12,Snowflake Inc - Class A
2,TSLA,200.45,188.71,202.27,0.00908,120831762,Mega-Cap,6.281041e+11,XNAS,Nasdaq All Markets,1.708045e+12,Tesla Inc


In [82]:
# # drop columns other than 'sybol', 'marketcap', 'exchange', 'exchangeName', 'marketcapType'
# screener_df = screener_df[['symbol', 'marketcap', 'exchange', 'exchangeName', 'marketcapType',]]

# screener_df.head(10)

In [83]:
# # DEV - API endpoing parameter availability. 
# schema_base_test_url = 'https://cloud.iexapis.com/stable'
# test_symbol = 'PLTR' # test ticker symbol
# schema_test_url = f"{schema_base_test_url}/stock/{test_symbol}/quote?schema=true&token={iex_key}"
# # GET request
# schema_test_response = requests.get(schema_test_url)

# print(json.dumps(schema_test_response.json(), indent=4))


IDEAS
1. Robust value score (RBV)
2. create data frames for each market cap
3. create a function that does math and querying based on rules / conditions
4. apply function to the market cap data sets

NOTES
1. "Fit" - 